# Run this notebook 
include complete calendar and public holidays

In [0]:
%pip install lightgbm

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)

In [0]:
%run /Users/ebizindavyi@delhaize.be/Data_preparation/Parse_xls_files

In [0]:
%run /Users/ebizindavyi@delhaize.be/Evaluation/Evaluation_functions

In [0]:
import pandas as pd
import numpy as np
import datetime as dt
import lightgbm as lgb
from reusable.featurestore.features_schoolholidays import school_holidays
from pyspark.sql import functions as f
import holidays



## Evaluation Params

In [0]:
actual_cols = ['Dry Actuals', 'Dry Actuals']
fc_cols = ['Dry Fc', 'LGBM_FC']
error_suffixes = ['Dry_WOW', 'Dry_LGBM']
error_to_calc = ["Bias", "AbsError", "SqError","RelAbsError"]

## Load inbound data

In [0]:
inbound_2022_dir = '/dbfs/mnt/dataplatform/acc/DataScience/sandbox/ebiz/Inbound_FC/data/input/Ecom-Ops-status_2022.xlsx'

In [0]:
inbound_2022_df = create_master_df(inbound_2022_dir)


In [0]:
calendar_2022 = pd.DataFrame({'Dates':pd.date_range(dt.datetime(2022,1,3), periods=363, freq='d')})
inbound_2022_complete = calendar_2022.merge(inbound_2022_df[['Dates','Dry Actuals']], on='Dates',how='left').fillna(0).copy()

## LightGBM


##Parameters

In [0]:
#lgbm parameter
backtest_start_date = pd.to_datetime('2022-01-03')
backtest_end_date =  pd.to_datetime('2022-12-31')

#shift periods to create features(lags)
shift_period_1 = 7
shift_period_2 = 14
shift_period_3 = 21

# Initialize lgbm parameters 
params = {
    'objective': 'regression',  
    'metric': 'rmse',           
    'boosting_type': 'gbdt',    
    # 'max_depth':2,      
}
# variables 
feature_col = ['x_7days', 'x_14days', 'x_21days']
pred_period=6
end_horizon = 7
fc_freq=7
training_period= 90

### Clean the data

removes all sundays and holidays in the complete dataset

In [0]:
shift_periods = [shift_period_1, shift_period_2, shift_period_3]
column_names = ['x_7days', 'x_14days', 'x_21days']

for i, period in enumerate(shift_periods):
    inbound_2022_dry[column_names[i]] = inbound_2022_dry['Dry Actuals'].shift(period)


In [0]:
inbound_2022_dry

,Dates,Dry Actuals,x_7days,x_14days,x_21days
0,2022-01-03,84224.0,NaN,NaN,NaN
1,2022-01-04,49564.0,NaN,NaN,NaN
2,2022-01-05,45890.0,NaN,NaN,NaN
3,2022-01-06,26976.0,NaN,NaN,NaN
4,2022-01-07,37044.0,NaN,NaN,NaN
...,...,...,...,...,...
358,2022-12-27,63764.0,75921.0,68514.0,76287.0
359,2022-12-28,47680.0,58697.0,65979.0,63301.0
360,2022-12-29,39552.0,62334.0,56286.0,54256.0
361,2022-12-30,61119.0,55510.0,50336.0,52050.0


In [0]:
#drop Nan and sundays and holidays with Zero in Actuals 
inbound_2022_dry = inbound_2022_dry.dropna().loc[inbound_2022_dry['Dry Actuals'] != 0].reset_index(drop=True)


In [0]:
inbound_2022_dry

,Dates,Dry Actuals,x_7days,x_14days,x_21days
0,2022-01-24,102266.0,81534.0,71540.0,84224.0
1,2022-01-25,80584.0,72886.0,63644.0,49564.0
2,2022-01-26,70974.0,53783.0,66433.0,45890.0
3,2022-01-27,67565.0,0.0,52802.0,26976.0
4,2022-01-28,57571.0,52025.0,43554.0,37044.0
...,...,...,...,...,...
275,2022-12-27,63764.0,75921.0,68514.0,76287.0
276,2022-12-28,47680.0,58697.0,65979.0,63301.0
277,2022-12-29,39552.0,62334.0,56286.0,54256.0
278,2022-12-30,61119.0,55510.0,50336.0,52050.0


### Backtest

In [0]:
backtest_results = pd.DataFrame()

inbound_2022_train = inbound_2022_dry[['Dates', 'Dry Actuals']].copy()#original non filtered dataset, 


# Iterate through each backtesting period
while backtest_start_date + dt.timedelta(days=pred_period) <= backtest_end_date:
  print(f"Backtesting on: {backtest_start_date}")
  
  start_train_date = backtest_start_date
  # end_train_date = backtest_start_date + dt.timedelta(days=training_period)
  end_train_date = start_train_date + dt.timedelta(days=training_period)
  print(f"Backtesting training ending on: {end_train_date}") 

  lgbm_train = inbound_2022_train[(inbound_2022_train['Dates'] >= start_train_date) &(inbound_2022_train['Dates'] < end_train_date)].copy()
  print(f"print the shape:{lgbm_train.shape} ")
  # Convert Dates column to datetime
  lgbm_train['Dates'] = pd.to_datetime(lgbm_train['Dates'])
    
  #Train features 
  train_features_df = inbound_2022_dry[(lgbm_train['Dates'] >= start_train_date) &(inbound_2022_dry['Dates'] < end_train_date)].copy()
  features_values_train=train_features_df.iloc[:, -3:]

  # Filter the dates with values in Dry Actuals 
  features_train = features_values_train.copy() 
  target = lgbm_train['Dry Actuals']
  
  # Create LightGBM dataset
  lgb_train = lgb.Dataset(features_train, label=target)
    
  # Model training
  m = lgb.train(params, lgb_train)

  # Prediction for the current period
  future_dates = pd.date_range(start=end_train_date+dt.timedelta(days=15), periods=pred_period, freq='D')

  future_features = pd.DataFrame({'ds': future_dates})
  future_features['ds'] = pd.to_datetime(future_features['ds'])

  future_features = inbound_2022_dry[inbound_2022_dry['Dates'].isin(future_dates)].iloc[:, -3:].values ### Changed the source df

  prediction_dates = inbound_2022_dry[inbound_2022_dry['Dates'].isin(future_dates)]['Dates'] ### New variable to extract valid dates from the future date range
  
  #forecast using lgbm model m

  forecast = m.predict(future_features, num_iteration=m.best_iteration,predict_disable_shape_check=True)

  lgbm_pred = pd.DataFrame({'ds': prediction_dates, 'yhat': forecast}) ### Use dates form df rather than the original date range
    
  backtest_results = pd.concat([backtest_results, lgbm_pred])
    
  # Increase training period after each iterations 
  backtest_start_date += dt.timedelta(days=end_horizon)
 
print("Backtesting completed.")


Backtesting on: 2022-01-03 00:00:00
Backtesting training ending on: 2022-04-03 00:00:00
print the shape:(58, 2) 
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000058 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 60
[LightGBM] [Info] Number of data points in the train set: 58, number of used features: 3
[LightGBM] [Info] Start training from score 66156.568966
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

In [0]:
backtest_results

,ds,yhat
69,2022-04-19,65070.847183
70,2022-04-20,51940.494917
71,2022-04-21,53930.894926
72,2022-04-22,53497.256396
73,2022-04-23,54557.543428
...,...,...
275,2022-12-27,83737.749143
276,2022-12-28,66844.019585
277,2022-12-29,53417.484211
278,2022-12-30,59988.541082


### Reformat backtest

In [0]:
backtest_results['Dates']= backtest_results['ds']
backtest_results['LGBM_FC']= backtest_results['yhat']
backtest_df = backtest_results[['Dates','LGBM_FC']].reset_index(drop=True).copy()
backtest_df['Dates'] = pd.to_datetime(backtest_df['Dates'])

### Join the backtest to the inbound

In [0]:
###Remove the data quality issues where FC is exactly te same as actuals

backtest_df_merge = pd.merge(inbound_2022_df[inbound_2022_df['Dry Fc']!=inbound_2022_df['Dry Actuals']],backtest_df, on="Dates", how='inner')[['Dates','Dry Actuals','Dry Fc','LGBM_FC']]
backtest_df_merge

,Dates,Dry Actuals,Dry Fc,LGBM_FC
0,2022-04-19,87006.0,81040.622023,65070.847183
1,2022-04-20,49021.0,58967.273526,51940.494917
2,2022-04-21,44875.0,59909.705493,53930.894926
3,2022-04-22,49405.0,57751.017683,53497.256396
4,2022-04-23,48878.0,54438.105073,54557.543428
...,...,...,...,...
206,2022-12-27,63764.0,71459.270000,83737.749143
207,2022-12-28,47680.0,71916.320000,66844.019585
208,2022-12-29,39552.0,60346.346000,53417.484211
209,2022-12-30,61119.0,52203.888000,59988.541082


## Evaluate

### Create error columns

In [0]:
for error in  error_to_calc:
  for i in range(len(actual_cols)):
    calculate_errors(df_eval= backtest_df_merge, actuals_col=actual_cols[i], fc_col= fc_cols[i],error_to_calculate=error,error_colname_suffix=error_suffixes[i])
    

In [0]:
backtest_df_merge

,Dates,Dry Actuals,Dry Fc,LGBM_FC,Bias_Dry_WOW,Bias_Dry_LGBM,AbsError_Dry_WOW,AbsError_Dry_LGBM,SqError_Dry_WOW,SqError_Dry_LGBM,RelAbsError_Dry_WOW,Outside_range_Dry_WOW,RelAbsError_Dry_LGBM,Outside_range_Dry_LGBM
0,2022-04-19,87006.0,81040.622023,65070.847183,-5965.377977,-21935.152817,5965.377977,21935.152817,3.558573e+07,4.811509e+08,0.068563,False,0.252111,True
1,2022-04-20,49021.0,58967.273526,51940.494917,9946.273526,2919.494917,9946.273526,2919.494917,9.892836e+07,8.523451e+06,0.202898,True,0.059556,False
2,2022-04-21,44875.0,59909.705493,53930.894926,15034.705493,9055.894926,15034.705493,9055.894926,2.260424e+08,8.200923e+07,0.335035,True,0.201803,True
3,2022-04-22,49405.0,57751.017683,53497.256396,8346.017683,4092.256396,8346.017683,4092.256396,6.965601e+07,1.674656e+07,0.168931,True,0.082831,False
4,2022-04-23,48878.0,54438.105073,54557.543428,5560.105073,5679.543428,5560.105073,5679.543428,3.091477e+07,3.225721e+07,0.113755,True,0.116198,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,2022-12-27,63764.0,71459.270000,83737.749143,7695.270000,19973.749143,7695.270000,19973.749143,5.921718e+07,3.989507e+08,0.120684,True,0.313245,True
207,2022-12-28,47680.0,71916.320000,66844.019585,24236.320000,19164.019585,24236.320000,19164.019585,5.873992e+08,3.672596e+08,0.508312,True,0.401930,True
208,2022-12-29,39552.0,60346.346000,53417.484211,20794.346000,13865.484211,20794.346000,13865.484211,4.324048e+08,1.922517e+08,0.525747,True,0.350563,True
209,2022-12-30,61119.0,52203.888000,59988.541082,-8915.112000,-1130.458918,8915.112000,1130.458918,7.947922e+07,1.277937e+06,0.145865,True,0.018496,False


### Calculate global metrics

In [0]:
global_metrics_dry = describe_inbound_fc(inbound_fc=backtest_df_merge, cols_to_keep=['Dry Actuals','Dry Fc','LGBM_FC','Bias_Dry_WOW','Bias_Dry_LGBM', 'AbsError_Dry_WOW','AbsError_Dry_LGBM','SqError_Dry_WOW','SqError_Dry_LGBM','RelAbsError_Dry_WOW','RelAbsError_Dry_LGBM', 'Outside_range_Dry_WOW', 'Outside_range_Dry_LGBM'])

<command-203330119889549>:25: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  described_df = inbound_fc.describe(include='all')[cols_to_keep].rename_axis('Metrics').reset_index(drop=False).copy()


In [0]:
global_metrics_dry

,Metrics,Dry Actuals,Dry Fc,LGBM_FC,Bias_Dry_WOW,Bias_Dry_LGBM,AbsError_Dry_WOW,AbsError_Dry_LGBM,SqError_Dry_WOW,SqError_Dry_LGBM,RelAbsError_Dry_WOW,RelAbsError_Dry_LGBM,Outside_range_Dry_WOW,Outside_range_Dry_LGBM
0,count,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,2.110000e+02,2.110000e+02,211.000000,211.000000,211,211
1,unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2
2,top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
3,freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122,142
4,first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,mean,65063.497630,66931.918290,65539.791821,1868.420660,476.294191,6792.152155,14135.819983,1.020815e+08,4.308162e+08,0.127793,0.225709,NaN,NaN
7,std,23712.525546,20937.319676,14976.821196,9952.887581,20799.995439,7497.638116,15234.658171,3.282153e+08,1.248045e+09,0.218364,0.275185,NaN,NaN
8,min,20444.000000,35712.572000,42238.225639,-59204.840000,-115044.574492,4.000000,103.795400,1.600000e+01,1.077349e+04,0.000081,0.001203,NaN,NaN
9,25%,49463.500000,54982.944347,53460.015293,-2795.068825,-7501.415797,2170.327353,4895.501604,4.710363e+06,2.396607e+07,0.033796,0.083451,NaN,NaN


### Visualize profiles

In [0]:
plot_two_inbound_fc(inbound_df=backtest_df_merge, actuals_col='Dry Actuals', fc_col_1='Dry Fc', fc_col_2= 'LGBM_FC', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')

### Visualize errors

In [0]:
plot_inbound_two_errors(inbound_df=backtest_df_merge, error_col_1='Bias_Dry_WOW', error_col_2= 'Bias_Dry_LGBM', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')

In [0]:
plot_inbound_two_errors(inbound_df=backtest_df_merge, error_col_1='AbsError_Dry_WOW', error_col_2= 'AbsError_Dry_LGBM', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')